# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` なし）
* `NOT IN` は未使用
* 判定は **ID 基準**、表示は仕様どおり `actor_id, director_id`

## 1) 問題

* `ある俳優 (actor_id) と監督 (director_id) の組み合わせで、協働回数が3回以上のペアを求める。`
* 入力テーブル例: `ActorDirector(actor_id INT, director_id INT, timestamp INT PRIMARY KEY)`
* 出力仕様: `actor_id, director_id`（重複なし・順不同）

---

## 2) 最適解（単一クエリ）

> ウィンドウ集計でペアごとの協働回数を数え、3回以上のみを射影。重複除去は `DISTINCT`。

```sql
WITH win AS (
  SELECT
    actor_id,
    director_id,
    COUNT(*) OVER (PARTITION BY actor_id, director_id) AS coop_cnt
  FROM ActorDirector
)
SELECT DISTINCT
  actor_id,
  director_id
FROM win
WHERE coop_cnt >= 3;

Runtime 349 ms
Beats 56.27%

```

* ポイント: `COUNT(*) OVER (PARTITION BY actor_id, director_id)` でペアごとの総件数を1パスで算出。
* 出力は仕様列のみ、順序指定なし。

---

## 3) 代替解

> 集約で十分に速いケース。実務ではこちらが最小コストになりやすい。

```sql
SELECT
  actor_id,
  director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3;

Runtime 348 ms
Beats 57.64%

```

* 追加の手段（参考・必要時のみ）: 事前にユニークな重複を除く必要があれば `SELECT DISTINCT actor_id, director_id, timestamp ...` の下位派生を作り `GROUP BY`。

---

## 4) 要点解説

* **方針**: ペア単位で出現回数をカウント → しきい値（3回）でフィルタ → 必要列のみ投影。
* **NULL/重複**: 入力列はINTでNULL前提なし、`timestamp` は主キーで重複なし。
* **インデックス**: `PRIMARY KEY(timestamp)` だけだとペア集計で全表走査になりがち。

  * 可能なら **複合インデックス `(actor_id, director_id)`** を作成すると `GROUP BY` / `PARTITION BY` の集約が効率化。

---

## 5) 計算量（概算）

* ウィンドウ解: 各パーティション集計で **O(N)**〜**O(N log N)**（実装・メモリアルゴ次第）。
* `GROUP BY` 解: ソート/ハッシュ集約で **O(N log N)** 近辺（適切なインデックスで実効はほぼ **O(N)**）。

---

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 テーブル]
  B[ペア単位の回数を算出]
  C[回数が3以上を抽出]
  D[出力 俳優ID 監督ID]
  A --> B
  B --> C
  C --> D
```

---

### 補足（実運用メモ）

* データ量が多い場合は `GROUP BY` 案＋ `(actor_id, director_id)` の複合インデックスが最も簡潔で速い構成になりやすいです。
* 結果順は任意のため、**`ORDER BY` を付けない**ことで不要なソートを省きます。

結論：**クエリ自体は最短経路**です。差が出るのは **実装より実行計画（インデックス・統計・メモリ）**。以下を順に打つと体感で大きく縮みます。

---

## 即効性のある改善（順に適用）

1. **複合インデックスを追加（最重要）**
   `GROUP BY actor_id, director_id` をインデックス順でなぞらせ、**ソート／テンポラリ回避**を狙います。

```sql
CREATE INDEX idx_actor_director ON ActorDirector (actor_id, director_id);
```

* これで `GROUP BY` 案はほぼ **インデックス順走査→集約** に変わります。
* 結果列がキーだけなので、**セカンダリインデックスのみ**で完結 (InnoDB はセカンダリ葉にPK含むが今回未参照)。

2. **ウィンドウ版は封印、`GROUP BY` を採用**
   ウィンドウ版は `COUNT OVER` → `DISTINCT` で無駄に行を増やします。最速はこれ：

```sql
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3;
```

> インデックスが効けば **ファイルソート/テンポラリなし** で流せます。

3. **統計の鮮度を上げる**

```sql
ANALYZE TABLE ActorDirector;
```

* 古い統計だとインデックスを握ってくれないことがあります。

4. **ハッシュ集約の挙動を確認（MySQL 8）**
   場合によってはハッシュ集約が一時領域を使い遅くなることがあります。悪化時のみヒントで切替。

```sql
-- 悪い計画が出たときだけ
SELECT /*+ NO_HASH_AGGREGATION() */
  actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3;
```

---

## 追加の選択肢（データ量・更新頻度しだい）

* **集計サマリ表**（マテビュー代替）

  ```sql
  -- 初期ロード
  CREATE TABLE CoopSummary AS
  SELECT actor_id, director_id, COUNT(*) AS coop_cnt
  FROM ActorDirector
  GROUP BY actor_id, director_id;

  CREATE UNIQUE INDEX ux_coop ON CoopSummary(actor_id, director_id);
  ```

  * 以降はバッチで増分反映（新規 `ActorDirector` のみ集計→`INSERT ... ON DUPLICATE KEY UPDATE`）。
  * 本問は「閾値3以上の存在判定」なので `WHERE coop_cnt >= 3` で**即時応答**。

---

## 期待できる効果（目安）

* 複合インデックス導入だけで、**中〜大規模**でも 2〜10倍短縮が珍しくありません。
* あなたの計測（~350ms）規模なら、**2桁〜100ms台**まで落ちる可能性が高いです（I/O・メモリ次第）。

---

## チェックリスト（必ず EXPLAIN）

```sql
EXPLAIN
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3;
```

* `key = idx_actor_director` が選ばれているか
* `Extra` に **Using index**、**Using temporary/Using filesort が消えているか**
* 行数見積り（rows）が現実的か（統計が効いているか）

---

## まとめ（処方箋）

* **ベースクエリ**：`GROUP BY ... HAVING COUNT(*) >= 3`
* **必須インデックス**：`(actor_id, director_id)`
* **統計更新**：`ANALYZE TABLE`
* （必要時のみ）**NO_HASH_AGGREGATION ヒント**
* 本番ワークロードでまだ重いなら、**集計サマリ表**で根本対策

この順で手当すれば、数字はまだ縮みます。ボトルネックはクエリじゃなく**計画**—です。
